# 数据格式转换

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_dataset_conversion.ipynb)&emsp;
[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_dataset_conversion.py)&emsp;
[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/intermediate/dataset_load_process/dataset_conversion.ipynb)

## 概述

用户可以将非标准数据集和常用数据集转换为MindSpore格式数据（MindRecord），从而可以方便地保存和加载数据。同时，MindSpore还针对部分场景做了性能优化，使用MindRecord数据格式可以减少磁盘IO、网络IO开销，从而获得更好的使用体验。

MindRecord数据格式具备的特征如下：

1. 实现多变的用户数据统一存储、访问，训练数据读取更加简便。
2. 数据聚合存储，高效读取，且方便管理、移动。
3. 高效的数据编解码操作，对用户透明、无感知。
4. 可以灵活控制分区的大小，实现分布式训练。

MindSpore数据格式的目标是归一化用户的数据集，并进一步通过`MindDataset`接口实现数据的读取，并用于训练过程。

![data_conversion_concept](https://gitee.com/mindspore/docs/raw/tutorials-develop/tutorials/source_zh_cn/intermediate/dataset_load_process/images/data_conversion_concept.png)

## MindRecord文件结构

一个MindRecord文件由数据文件和索引文件组成。数据文件包含文件头、标量数据页、块数据页，用于存储用户归一化后的训练数据，且单个MindRecord文件建议小于20G，用户可将大数据集进行分片存储为多个MindRecord文件；索引文件包含基于标量数据（如图像Label、图像文件名等）生成的索引信息，用于方便的检索、统计数据集信息。

文件头、标量数据页、块数据页的用途如下：

- 文件头：主要用来存储文件头大小、标量数据页大小、块数据页大小、Schema信息、索引字段、统计信息、文件分区信息、标量数据与块数据对应关系等，是MindRecord文件的元信息。
- 标量数据页：主要用来存储整型、字符串、浮点型数据，如图像的Label、图像的文件名、图像的长宽等信息，即适合用标量来存储的信息会保存在这里。
- 块数据页：主要用来存储二进制串、NumPy数组等数据，如二进制图像文件本身、文本转换成的字典等。

数据文件和索引文件均暂不支持重命名操作，其示意图如下：

![mindrecord](https://gitee.com/mindspore/docs/raw/tutorials-develop/tutorials/source_zh_cn/intermediate/dataset_load_process/images/mindrecord.png)

## 非标准数据集转换MindRecord

下面主要介绍如何将CV类数据和NLP类数据转换为MindRecord，并通过`MindDataset`接口实现MindRecord文件的读取。

### 转换CV类数据集

本示例主要介绍用户如何将自己的CV类数据集转换成MindRecord，并使用`MindDataset`接口读取。

首先创建一个包含100条记录的CV数据集并将其转换为MindRecord格式后保存，其样本包含`file_name`（字符串）、`label`（整型）、 `data`（二进制）三个字段，然后使用`MindDataset`接口读取该MindRecord文件。

1. 导入相关模块。

In [27]:
from io import BytesIO
import os
import mindspore.dataset as ds
from mindspore.mindrecord import FileWriter
import mindspore.dataset.vision.c_transforms as vision
from PIL import Image

2. 生成100张图像，并转换成MindRecord。

In [28]:
MINDRECORD_FILE = "test.mindrecord"

if os.path.exists(MINDRECORD_FILE):
    os.remove(MINDRECORD_FILE)
    os.remove(MINDRECORD_FILE + ".db")

writer = FileWriter(file_name=MINDRECORD_FILE, shard_num=1)

cv_schema = {"file_name": {"type": "string"}, "label": {"type": "int32"}, "data": {"type": "bytes"}}
writer.add_schema(cv_schema, "it is a cv dataset")

writer.add_index(["file_name", "label"])

data = []
for i in range(100):
    i += 1
    sample = {}
    white_io = BytesIO()
    Image.new('RGB', (i*10, i*10), (255, 255, 255)).save(white_io, 'JPEG')
    image_bytes = white_io.getvalue()
    sample['file_name'] = str(i) + ".jpg"
    sample['label'] = i
    sample['data'] = white_io.getvalue()

    data.append(sample)
    if i % 10 == 0:
        writer.write_raw_data(data)
        data = []

if data:
    writer.write_raw_data(data)

writer.commit()

<MSRStatus.SUCCESS: 0>

参数说明：

- `MINDRECORD_FILE`：输出的MindRecord文件路径。

3. 通过`MindDataset`接口读取MindRecord。

In [29]:
data_set = ds.MindDataset(dataset_files=MINDRECORD_FILE)
decode_op = vision.Decode()
data_set = data_set.map(operations=decode_op, input_columns=["data"], num_parallel_workers=2)
count = 0
for item in data_set.create_dict_iterator(output_numpy=True):
    count += 1
print("Got {} samples".format(count))

Got 100 samples


### 转换NLP类数据集

本示例主要介绍用户如何将自己的NLP类数据集转换成MindRecord，并使用`MindDataset`接口读取。为了方便展示，此处略去了将文本转换成字典序的预处理过程。

本示例首先创建一个包含100条记录的MindRecord文件，其样本包含八个字段，均为整型数组，然后使用`MindDataset`接口读取该MindRecord文件。

1. 导入相关模块。

In [30]:
import os
import numpy as np
import mindspore.dataset as ds
from mindspore.mindrecord import FileWriter

2. 生成100条文本数据，并转换成MindRecord。

In [31]:
MINDRECORD_FILE = "test.mindrecord"

if os.path.exists(MINDRECORD_FILE):
    os.remove(MINDRECORD_FILE)
    os.remove(MINDRECORD_FILE + ".db")

writer = FileWriter(file_name=MINDRECORD_FILE, shard_num=1)

nlp_schema = {"source_sos_ids": {"type": "int64", "shape": [-1]},
              "source_sos_mask": {"type": "int64", "shape": [-1]},
              "source_eos_ids": {"type": "int64", "shape": [-1]},
              "source_eos_mask": {"type": "int64", "shape": [-1]},
              "target_sos_ids": {"type": "int64", "shape": [-1]},
              "target_sos_mask": {"type": "int64", "shape": [-1]},
              "target_eos_ids": {"type": "int64", "shape": [-1]},
              "target_eos_mask": {"type": "int64", "shape": [-1]}}
writer.add_schema(nlp_schema, "it is a preprocessed nlp dataset")

data = []
for i in range(100):
    i += 1

    sample = {"source_sos_ids": np.array([i, i + 1, i + 2, i + 3, i + 4], dtype=np.int64),
              "source_sos_mask": np.array([i * 1, i * 2, i * 3, i * 4, i * 5, i * 6, i * 7], dtype=np.int64),
              "source_eos_ids": np.array([i + 5, i + 6, i + 7, i + 8, i + 9, i + 10], dtype=np.int64),
              "source_eos_mask": np.array([19, 20, 21, 22, 23, 24, 25, 26, 27], dtype=np.int64),
              "target_sos_ids": np.array([28, 29, 30, 31, 32], dtype=np.int64),
              "target_sos_mask": np.array([33, 34, 35, 36, 37, 38], dtype=np.int64),
              "target_eos_ids": np.array([39, 40, 41, 42, 43, 44, 45, 46, 47], dtype=np.int64),
              "target_eos_mask": np.array([48, 49, 50, 51], dtype=np.int64)}

    data.append(sample)
    if i % 10 == 0:
        writer.write_raw_data(data)
        data = []

if data:
    writer.write_raw_data(data)

writer.commit()

<MSRStatus.SUCCESS: 0>

参数说明：

- `MINDRECORD_FILE`：输出的MindRecord文件路径。

3. 通过`MindDataset`接口读取MindRecord。

In [32]:
data_set = ds.MindDataset(dataset_files=MINDRECORD_FILE)
count = 0
for item in data_set.create_dict_iterator():
    count += 1
print("Got {} samples".format(count))

Got 100 samples


## 常用数据集转换MindRecord

MindSpore提供转换常用数据集的工具类，能够将常用的数据集转换为MindRecord。部分常用数据集及其对应的工具类列表如下，更多数据集转换的详细说明参考[API文档](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.mindrecord.html)。

| 数据集 | 格式转换工具类 |
| :------- | :----------- |
| CIFAR-10 | Cifar10ToMR  |
| ImageNet | ImageNetToMR |
| TFRecord | TFRecordToMR |
| CSV File | CsvToMR |

### 转换CIFAR-10数据集

用户可以通过`Cifar10ToMR`类，将CIFAR-10原始数据转换为MindRecord，并使用`MindDataset`接口读取。

1. 下载[CIFAR-10数据集](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz)并解压到指定目录，以下示例代码将数据集下载并解压到指定位置。

In [33]:
from mindvision.dataset import DownLoad
dl = DownLoad()
dl_url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/datasets/cifar-10-python.tar.gz"
dl_path = "./datasets"
dl.download_and_extract_archive(url=dl_url, download_path=dl_path)

解压后数据集文件的目录结构如下：

```text
./datasets/cifar-10-batches-py
├── batches.meta
├── data_batch_1
├── data_batch_2
├── data_batch_3
├── data_batch_4
├── data_batch_5
├── readme.html
└── test_batch
```

2. 导入相关模块。

In [34]:
import os
import mindspore.dataset as ds
import mindspore.dataset.vision.c_transforms as vision
from mindspore.mindrecord import Cifar10ToMR

3. 创建`Cifar10ToMR`对象，调用`transform`接口，将CIFAR-10数据集转换为MindRecord。

In [35]:
ds_target_path = "./datasets/mindspore_dataset_conversion/"

os.system("rm -f {}*".format(ds_target_path))
os.system("mkdir -p {}".format(ds_target_path))

CIFAR10_DIR = "./datasets/cifar-10-batches-py"
MINDRECORD_FILE = "./datasets/mindspore_dataset_conversion/cifar10.mindrecord"
cifar10_transformer = Cifar10ToMR(CIFAR10_DIR, MINDRECORD_FILE)
cifar10_transformer.transform(['label'])

<MSRStatus.SUCCESS: 0>

参数说明：

- `CIFAR10_DIR`：CIFAR-10数据集路径。
- `MINDRECORD_FILE`：输出的MindRecord文件路径。

4. 通过`MindDataset`接口读取MindRecord。

In [36]:
data_set = ds.MindDataset(dataset_files=MINDRECORD_FILE)
decode_op = vision.Decode()
data_set = data_set.map(operations=decode_op, input_columns=["data"], num_parallel_workers=2)
count = 0
for item in data_set.create_dict_iterator(output_numpy=True):
    count += 1
print("Got {} samples".format(count))

Got 50000 samples


### 转换CSV数据集

本示例首先创建一个包含5条记录的CSV文件，然后通过`CsvToMR`工具类将CSV文件转换为MindRecord，并最终通过`MindDataset`接口将其读取出来。

1. 导入相关模块。

In [37]:
import csv
import os
import mindspore.dataset as ds
from mindspore.mindrecord import CsvToMR

2. 生成CSV文件，并转换成MindRecord。

In [38]:
CSV_FILE = "test.csv"
MINDRECORD_FILE = "test.mindrecord"

def generate_csv():
    headers = ["id", "name", "math", "english"]
    rows = [(1, "Lily", 78.5, 90),
            (2, "Lucy", 99, 85.2),
            (3, "Mike", 65, 71),
            (4, "Tom", 95, 99),
            (5, "Jeff", 85, 78.5)]
    with open(CSV_FILE, 'w', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(rows)

generate_csv()

if os.path.exists(MINDRECORD_FILE):
    os.remove(MINDRECORD_FILE)
    os.remove(MINDRECORD_FILE + ".db")

csv_transformer = CsvToMR(CSV_FILE, MINDRECORD_FILE, partition_number=1)

csv_transformer.transform()

assert os.path.exists(MINDRECORD_FILE)
assert os.path.exists(MINDRECORD_FILE + ".db")

参数说明：

- `CSV_FILE`：CSV文件的路径。
- `MINDRECORD_FILE`：输出的MindRecord文件路径。

3. 通过`MindDataset`接口读取MindRecord。

In [39]:
data_set = ds.MindDataset(dataset_files=MINDRECORD_FILE)
count = 0
for item in data_set.create_dict_iterator(output_numpy=True):
    count += 1
print("Got {} samples".format(count))

Got 5 samples


### 转换TFRecord数据集

本示例需提前安装TensorFlow，目前只支持TensorFlow 1.13.0-rc1及以上版本。如本文档以Notebook运行时，完成安装后需要重启kernel才能执行后续代码。

本示例首先通过TensorFlow创建一个TFRecord文件，然后通过`TFRecordToMR`工具类将TFRecord文件转换为MindRecord，最后通过`MindDataset`接口将其读取出来，并使用`Decode`函数对`image_bytes`字段进行解码。

1. 导入相关模块。

In [40]:
import collections
from io import BytesIO
import os
import mindspore.dataset as ds
from mindspore.mindrecord import TFRecordToMR
import mindspore.dataset.vision.c_transforms as vision
from PIL import Image
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2. 生成TFRecord文件。

In [41]:
TFRECORD_FILE = "test.tfrecord"
MINDRECORD_FILE = "test.mindrecord"

def generate_tfrecord():
    def create_int_feature(values):
        if isinstance(values, list):
            feature = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
        else:
            feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        return feature

    def create_float_feature(values):
        if isinstance(values, list):
            feature = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
        else:
            feature = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        return feature

    def create_bytes_feature(values):
        if isinstance(values, bytes):
            white_io = BytesIO()
            Image.new('RGB', (10, 10), (255, 255, 255)).save(white_io, 'JPEG')
            image_bytes = white_io.getvalue()
            feature = tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))
        else:
            feature = tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(values, encoding='utf-8')]))
        return feature

    writer = tf.io.TFRecordWriter(TFRECORD_FILE)

    example_count = 0
    for i in range(10):
        file_name = "000" + str(i) + ".jpg"
        image_bytes = bytes(str("aaaabbbbcccc" + str(i)), encoding="utf-8")
        int64_scalar = i
        float_scalar = float(i)
        int64_list = [i, i+1, i+2, i+3, i+4, i+1234567890]
        float_list = [float(i), float(i+1), float(i+2.8), float(i+3.2),
                      float(i+4.4), float(i+123456.9), float(i+98765432.1)]

        features = collections.OrderedDict()
        features["file_name"] = create_bytes_feature(file_name)
        features["image_bytes"] = create_bytes_feature(image_bytes)
        features["int64_scalar"] = create_int_feature(int64_scalar)
        features["float_scalar"] = create_float_feature(float_scalar)
        features["int64_list"] = create_int_feature(int64_list)
        features["float_list"] = create_float_feature(float_list)

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
        example_count += 1
    writer.close()
    print("Write {} rows in tfrecord.".format(example_count))

generate_tfrecord()

Write 10 rows in tfrecord.


参数说明：

- `TFRECORD_FILE`：TFRecord文件的路径。
- `MINDRECORD_FILE`：输出的MindRecord文件路径。

3. 将TFRecord转换成MindRecord。

In [42]:
feature_dict = {"file_name": tf.io.FixedLenFeature([], tf.string),
                "image_bytes": tf.io.FixedLenFeature([], tf.string),
                "int64_scalar": tf.io.FixedLenFeature([], tf.int64),
                "float_scalar": tf.io.FixedLenFeature([], tf.float32),
                "int64_list": tf.io.FixedLenFeature([6], tf.int64),
                "float_list": tf.io.FixedLenFeature([7], tf.float32),
                }

if os.path.exists(MINDRECORD_FILE):
    os.remove(MINDRECORD_FILE)
    os.remove(MINDRECORD_FILE + ".db")

tfrecord_transformer = TFRecordToMR(TFRECORD_FILE, MINDRECORD_FILE, feature_dict, ["image_bytes"])
tfrecord_transformer.transform()

assert os.path.exists(MINDRECORD_FILE)
assert os.path.exists(MINDRECORD_FILE + ".db")

4. 通过`MindDataset`接口读取MindRecord。

In [43]:
data_set = ds.MindDataset(dataset_files=MINDRECORD_FILE)
decode_op = vision.Decode()
data_set = data_set.map(operations=decode_op, input_columns=["image_bytes"], num_parallel_workers=2)
count = 0
for item in data_set.create_dict_iterator(output_numpy=True):
    count += 1
print("Got {} samples".format(count))

Got 10 samples
